In [1]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import WebDriverWait
import bs4
import pandas as pd
import re
import time


In [2]:
# driver_path = 'C:/Users/jaehkim/Desktop/sermon_analysis/chromedriver.exe'
driver_path = '/Users/hyun-young/workspace/github/chromedriver'
url = 'https://www.fgtv.com/fgtv/f1/01.asp'

driver = webdriver.Chrome(driver_path)
driver.implicitly_wait(3)

driver.get(url)

# FNGTV 방송 - 예배 0~9 Loop

In [8]:
# left_menu_titles : 한 번 돌아갈떄마다 바뀌므로 동시에 실행
left_menu_titles = driver.find_element_by_class_name("depth2_gap")

worships = left_menu_titles.text.split('\n') #categories
buttons = left_menu_titles.find_elements_by_tag_name('a') 
for button in buttons:
    button.click()
    driver.implicitly_wait(3)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=84.0.4147.105)


# 1부~7부 Loop

In [17]:
mainframe = driver.find_element_by_class_name("mainframe")
worship_times = mainframe.find_elements_by_tag_name('li')

for worship_time in worship_times:
    if re.search('\d부',worship_time.text):
        print(worship_time.text)
        #driver.implicitly_wait(3) 시도(1)
        #time.sleep(5) #시도(2)
        #wait = WebDriverWait(driver, 20) #시도(3)
        #element=wait.until(EC.presence_of_element_located((By.ID,worship_time.id)))
        #element.click()
    

1부
2부
3부
4부
5부(청년연합예배)
6부
7부


# Table

In [4]:
current_page = 1
rows= []

while(True):
    
    paging = driver.find_elements_by_class_name('table_f1_list')
    print("current:", paging[0].find_element_by_class_name('bbs02').text, current_page)
    
    page_buttons = paging[0].find_elements_by_class_name('bbs01')
    print(page_buttons[0].text)
    
    for buttons in page_buttons:
        if int(buttons.text) != current_page+1:
            continue
        
        tables = driver.find_elements_by_class_name('table_f1')
        if current_page == 1:
            table_columns = tables[0].text.split()
        row_element = tables[1].find_elements_by_tag_name('td')

        row = []
        for idx,x in enumerate(row_element):
            if idx%7 == 6:
                rows.append(row)
                row = []
                continue
#             print(idx,x.text)
            row.append(x.text)
        break
    
    if (current_page + 1) % 10 == 0:
        driver.find_element_by_class_name('table_f1_list_right').click()
        driver.implicitly_wait(3)
    else:
        print("page button click")
        buttons.click()
        driver.implicitly_wait(3)
        
    current_page+=1
    
    
    


current: 1 1
2
page button click
current: 1 2
2
page button click


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=85.0.4183.83)


0 2020-08-23
1 이장균 목사
2 코로나 시대와 슬기로운 신앙생활
3 눅 3:4~6
4 고화질
5 
7 2020-08-16
8 이장균 목사
9 믿음으로 아브라함은
10 히 11:8~10
11 고화질
12 
14 2020-08-09
15 이장균 목사
16 믿음으로 노아는
17 히 11:7
18 고화질
19 
21 2020-08-02
22 이장균 목사
23 믿음으로 에녹은
24 히 11:5
25 고화질
26 
28 2020-07-26
29 이장균 목사
30 믿음으로 아벨은
31 히 11:4
32 고화질
33 
35 2020-07-19
36 이영훈 목사
37 주님께서 나를 구원하시리로다
38 시 55:16~18, 22
39 고화질
40 
42 2020-07-12
43 이영훈 목사
44 성령충만의 4단계
45 겔 47:1~5
46 고화질
47 
49 2020-07-05
50 이영훈 목사
51 광야길 40년
52 출 14:13~14
53 고화질
54 
56 2020-06-28
57 이영훈 목사
58 주님을 사랑하라
59 시 31:23~24
60 고화질
61 
63 2020-06-21
64 이영훈 목사
65 상한 갈대를 꺾지 아니하며
66 마 12:17~21
67 고화질
68 


In [12]:
pd.DataFrame(data=rows,columns=table_columns)

,날짜,설교자,제목,본문,동영상,선택
0,2020-08-23,이장균 목사,코로나 시대와 슬기로운 신앙생활,눅 3:4~6,고화질,
1,2020-08-16,이장균 목사,믿음으로 아브라함은,히 11:8~10,고화질,
2,2020-08-09,이장균 목사,믿음으로 노아는,히 11:7,고화질,
3,2020-08-02,이장균 목사,믿음으로 에녹은,히 11:5,고화질,
4,2020-07-26,이장균 목사,믿음으로 아벨은,히 11:4,고화질,
5,2020-07-19,이영훈 목사,주님께서 나를 구원하시리로다,"시 55:16~18, 22",고화질,
6,2020-07-12,이영훈 목사,성령충만의 4단계,겔 47:1~5,고화질,
7,2020-07-05,이영훈 목사,광야길 40년,출 14:13~14,고화질,
8,2020-06-28,이영훈 목사,주님을 사랑하라,시 31:23~24,고화질,
9,2020-06-21,이영훈 목사,상한 갈대를 꺾지 아니하며,마 12:17~21,고화질,


# 페이지 넘기기

'2'

AttributeError: 'WebElement' object has no attribute 'txt'